In [1]:
# if running in Colab run this first, otherwise make sure flair[word-embeddings] is installed
import torch
!pip install flair[word-embeddings]
import flair
# it looks like we have to restart the Colab runtime after installation

In [3]:
import os
from pathlib import Path

from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, CharacterEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer


if __name__ == "__main__":
    # load corpus
    columns = {0: 'text', 1: 'ner'}
    data_folder = '/home/dataset' # upload dataset to this folder in Colab
    corpus: Corpus = ColumnCorpus(data_folder, columns,
                                train_file='train.txt',
                                test_file='test.txt',
                                dev_file='dev.txt',
                                column_delimiter='\t')

    # extract the labels from the corpus
    label_type = 'ner'
    label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)

    # train model
    embedding_types = [
        WordEmbeddings('twitter'), # tried 'glove', better results with 'twitter'
        FlairEmbeddings('news-forward'),
        FlairEmbeddings('news-backward')
    ]

    embeddings = StackedEmbeddings(embeddings=embedding_types)

    tagger = SequenceTagger(hidden_size=256,
                            embeddings=embeddings,
                            tag_dictionary=label_dict,
                            tag_type=label_type,
                            tag_format="BIO")

    trainer = ModelTrainer(tagger, corpus)

    # tuning learning rate
    learning_rates = [0.01, 0.005, 0.0025, 0.001]
    for lr in learning_rates:
        path = '/home/model-lr-' + str(lr)
        trainer.train(path,
                      learning_rate=lr,
                      mini_batch_size=32,
                      max_epochs=25)


2025-05-01 14:57:24,406 Reading data from /home/dataset
2025-05-01 14:57:24,407 Train: /home/dataset/train.txt
2025-05-01 14:57:24,408 Dev: /home/dataset/dev.txt
2025-05-01 14:57:24,410 Test: /home/dataset/test.txt
2025-05-01 14:57:26,364 Computing label dictionary. Progress:


4it [00:00, 5121.25it/s]
6338it [00:00, 39848.75it/s]

2025-05-01 14:57:26,534 Dictionary created for label 'ner' with 3 values: PER (seen 3101 times), ORG (seen 2267 times), LOC (seen 1996 times)


2025-05-01 14:57:43,940 SequenceTagger predicts: Dictionary with 7 tags: O, B-PER, I-PER, B-ORG, I-ORG, B-LOC, I-LOC
2025-05-01 14:57:44,132 ----------------------------------------------------------------------------------------------------
2025-05-01 14:57:44,133 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'twitter'
      (embedding): Embedding(1193515, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(41

/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


...

100%|██████████| 16/16 [00:03<00:00,  4.26it/s]

2025-05-01 15:26:38,461 DEV : loss 0.12236926704645157 - f1-score (micro avg)  0.7009


2025-05-01 15:26:38,531  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.0005]
2025-05-01 15:26:38,532  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.00025]
2025-05-01 15:26:38,533  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.000125]
2025-05-01 15:26:38,534  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [6.25e-05]
2025-05-01 15:26:38,535 ----------------------------------------------------------------------------------------------------
2025-05-01 15:26:38,536 learning rate too small - quitting training!
2025-05-01 15:26:38,537 ----------------------------------------------------------------------------------------------------
2025-05-01 15:26:38,538 Saving model ...
2025-05-01 15:26:55,929 Done.
2025-05-01 15:26:55,930 ----------------------------------------------------------------------------------------------------
2025-05-01 15:26:55,931 Load

100%|██████████| 32/32 [00:14<00:00,  2.26it/s]

2025-05-01 15:27:14,310 
Results:
- F-score (micro) 0.7132
- F-score (macro) 0.6452
- Accuracy 0.6028

By class:
              precision    recall  f1-score   support

         PER     0.8344    0.8458    0.8400      1602
         ORG     0.5978    0.4129    0.4884       792
         LOC     0.7428    0.5133    0.6071       602

   micro avg     0.7696    0.6646    0.7132      2996
   macro avg     0.7250    0.5907    0.6452      2996
weighted avg     0.7534    0.6646    0.7003      2996

2025-05-01 15:27:14,311 ----------------------------------------------------------------------------------------------------
